In [ ]:
!pip install fastdtw

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fastdtw: filename=fastdtw-0.3.4-cp310-cp310-linux_x86_64.whl size=512412 sha256=55ae48887f49f45654d4ed70df9d049e8d6116b4ee2e87e1fdb8d35e089ba7a5
  Stored in directory: /root/.cache/pip/wheels/73/c8/f7/c25448dab74c3acf4848bc25d513c736bb93910277e1528ef4
Successfully built fastdtw


In [ ]:
import numpy as np
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
class dongha_mingi_model:

    def __init__(self, last, last_gdp, last_con, this = None):
        self.last = last
        self.last_gdp = last_gdp
        self.last_con = last_con
        self.forecast_last = []
        self.alpha = 0
        self.beta = 0
        self.this = this
        self.this_arr = []
        self.threshold = None

    def fit(self, alpha, beta, threshold = None):
        self.alpha = alpha
        self.beta = beta
        self.threshold = threshold
        this = self.this[['Date', 'Close']].copy()
        last = self.last[['Date', 'Close']].copy()
        last_arr = last.iloc[:, 1].to_numpy().reshape(-1, 1)
        last_gdp_arr = self.last_gdp.reshape(-1, 1).copy()
        last_con_arr = self.last_con.reshape(-1, 1).copy()
        self.this_arr = this.iloc[:, 1].to_numpy().reshape(-1, 1)

        last_max = np.max(last_arr)
        last_min = np.min(last_arr)
        last_gdp_max = np.max(last_gdp_arr)
        last_gdp_min = np.min(last_gdp_arr)
        last_con_max = np.max(last_con_arr)
        last_con_min = np.min(last_con_arr)
        this_max = np.max(self.this_arr)
        this_min = np.min(self.this_arr)


        for i in range(len(last_arr)):
            last_arr[i] = (last_arr[i] - last_min) / (last_max - last_min)

        for i in range(len(last_gdp_arr)):
            last_gdp_arr[i] = (last_gdp_arr[i] - last_gdp_min) / (last_gdp_max - last_gdp_min)

        for i in range(len(last_con_arr)):
            last_con_arr[i] = (last_con_arr[i] - last_con_min) / (last_con_max - last_con_min)

        for i in range(len(self.this_arr)):
            self.this_arr[i] = (self.this_arr[i] - this_max) / (this_max - this_min)

        gdp_distance, gdp_path = fastdtw(last_gdp_arr, last_arr, dist=euclidean)
        con_distance, con_path = fastdtw(last_con_arr, last_arr, dist=euclidean)

        last_ratio = 0
        gdp_ratio = gdp_distance / (gdp_distance + con_distance)
        con_ratio = con_distance / (gdp_distance + con_distance)

        last_weight = 1 - last_ratio
        gdp_weight = 1 - gdp_ratio
        con_weight = 1 - con_ratio

        weight_list = [last_weight, gdp_weight, con_weight]


        if threshold is None :
          pass
        elif threshold is not None :
          for i in range(len(weight_list)):
            if weight_list[i] < threshold :
               weight_list[i] = 0

        result = []

        for i in range(len(last_arr)):
            result.append(last_arr[i] * weight_list[0] + last_gdp_arr[i] * weight_list[1] + last_con_arr[i] * weight_list[2])

        def double_exponential_smoothing(data, alpha, beta):
            level = data[0]
            trend = 0
            forecast = [data[0]]

            for t in range(1, len(data)) :
              prev_level = level
              prev_trend = trend

              level = alpha * data[t] + (1 - alpha) * (prev_level + prev_trend)
              trend = beta * (level - prev_level) + (1 - beta) * prev_trend

              forecast_t = level + trend
              forecast.append(forecast_t)

            return forecast

        forecast = double_exponential_smoothing(result, alpha, beta)

        self.forecast_last = []

        for i in range(len(forecast)):
           self.forecast_last.append(forecast[i][0])



    def getValue(self):
      if self.threshold is None:
        return np.array([self.alpha, self.beta])
      elif self.threshold is not None:
        return np.array([self.alpha, self.beta, self.threshold])

    def loss(self) :
        return np.sum((((self.this_arr - self.forecast_last)**2)/len(self.this)) / np.linalg.norm(self.this_arr))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
kospi2022 = pd.read_csv("/content/drive/MyDrive/캡스톤디자인/KOSPI2022.csv")
last_gdp =np.array([512012.9, 512012.9, 512012.9, 544239.8, 544239.8, 544239.8, 548892.5, 548892.5, 548892.5, 556628.7, 556628.7, 556628.7])
last_con = pd.read_csv("/content/drive/MyDrive/캡스톤디자인/2022소비자물가지수.csv")
last_con = last_con.iloc[0, :]
last_con = last_con[1:].to_numpy()
kospi2023 = pd.read_csv("/content/drive/MyDrive/캡스톤디자인/KOSPI_monthly.csv")

In [ ]:
model1 = dongha_mingi_model(last = kospi2022, last_gdp = last_gdp, last_con = last_con, this = kospi2023)

In [ ]:
model1.fit(alpha = 0.3, beta = 0.3)

In [ ]:
model1.forecast_last

[0.8433807828774889,
 0.8752612685543112,
 0.974607149412655,
 1.1892224983596853,
 1.4091505152428245,
 1.4007692162890701,
 1.488565792610659,
 1.5053547868142996,
 1.287002009791621,
 1.2132488689938674,
 1.206274683092365,
 1.0512749700613067]

In [ ]:
model1.getValue()

array([0.3, 0.3])

In [ ]:
model2 = dongha_mingi_model(last = kospi2022, last_gdp = last_gdp, last_con = last_con, this = kospi2023)

In [ ]:
model2.fit(alpha = 0.3, beta = 0.3, threshold = 0.5)

In [ ]:
model2.forecast_last

[0.8433807828774889,
 0.8752612685543112,
 0.974607149412655,
 1.0765694163433106,
 1.2017821881157595,
 1.1197665019452556,
 1.1390483028586027,
 1.1095834132195896,
 0.8647222259960081,
 0.7536254274148035,
 0.7258860211993884,
 0.5635948639113463]

In [ ]:
model2.getValue()

array([0.3, 0.3, 0.5])

In [ ]:
model1.loss()

18.4595831399838

In [ ]:
model2.loss()

13.166779809241188